In [1]:
import langchain
from langchain_community.document_loaders import DataFrameLoader
import json
import pandas as pd
import getpass
import os
from pinecone import Pinecone, ServerlessSpec
from langchain_pinecone import PineconeVectorStore
from langchain_openai import OpenAIEmbeddings 
import openai 
from langchain_core.documents import Document

c:\Users\lucas\.pyenv\pyenv-win\versions\3.11.0\Lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [16]:
departments = ['SWA', 'ARCTIC', 'TAGLG', 'AAS', 'CHSTU', 'BENG', 'CFRM', 'AFRAM', 'AES', 'ASIAN',
 'AMATH', 'ART', 'DESIGN', 'AIS', 'INDO', 'INDN', 'ARCHY', 'URDU', 'HINDI', 'SNKRT',
 'CHIN', 'ASTBIO', 'KOREAN', 'HUM', 'VIET', 'JAPAN', 'ASTR', 'CMS', 'GREEK', 'CLAS',
 'LATIN', 'DXARTS', 'ANTH', 'INDIV', 'CHID', 'CHEM', 'TXTDS', 'BIOL', 'DANCE',
 'COM', 'FRENCH', 'ITAL', 'HSTAFM', 'DRAMA', 'CSE', 'ECON', 'HSTLAC', 'HSTAM',
 'HPS', 'HSTRY', 'GWSS', 'HSTAS', 'HSTCMP', 'LABOR', 'GEOG', 'ASL', 'HSTEU',
 'GERMAN', 'RELIG', 'ARAMIC', 'COPTIC', 'GEEZ', 'EGYPT', 'LADINO', 'MICROM', 'LSJ',
 'JSIS', 'ARAB', 'TURKIC', 'CHGTAI', 'LING', 'BIBHEB', 'MODHEB', 'PRSAN', 'MATH',
 'ENGL', 'KYRGYZ', 'KAZAKH', 'UGARIT', 'UZBEK', 'UYGUR', 'TKISH', 'MUSAP', 'MUSED',
 'NEUSCI', 'ETHICS', 'MUSEN', 'MUHST', 'MELC', 'FINN', 'DANISH', 'ESTO', 'LITH',
 'LATV', 'NORW', 'BCMS', 'SWED', 'CZECH', 'BULGR', 'MUSICP', 'ROMN', 'PHIL',
 'POLSH', 'GLITS', 'UKR', 'SLVN', 'PORT', 'SCAND', 'PHYS', 'SPLING', 'MUSIC',
 'SLAVIC', 'RUSS', 'PSYCH', 'CEP', 'CM', 'SPHSC', 'STAT', 'ACCTG', 'ARCH', 'ENTRE',
 'SOC', 'FIN', 'SPAN', 'QMETH', 'OPMGT', 'MGMT', 'DPHS', 'MKTG', 'ORALM', 'OHS',
 'ORTHO', 'ECE', 'PERIO', 'ECFS', 'EDUC', 'NME', 'EDSPE', 'CEE', 'EDPSY', 'EDLPS',
 'ENGR', 'HCDE', 'BSE', 'FISH', 'FHL', 'MARBIO', 'ENVIR', 'SMEA', 'GRDSCH', 'ESRM',
 'INFO', 'OCEAN', 'ESS', 'LAW', 'NUTR', 'UCONJ', 'BIOC', 'ANEST', 'BIME', 'BIOEN',
 'IMMUN', 'GENOME', 'CONJ', 'PATH', 'MED', 'FAMED', 'OPHTH', 'OTOHN', 'ORTHP',
 'PHCOL', 'IECMH', 'RADGY', 'UROL', 'PBSCI', 'NEURL', 'SURG', 'NMETH', 'NCLIN',
 'PEDS', 'REHAB', 'MEDCH', 'PCEUT', 'PHARM', 'NURS', 'PUBPOL', 'HIHIM', 'BIOST',
 'EPI', 'PABIO', 'PHG', 'SPH', 'LEAD', 'HONORS', 'HSERV']


print(len(departments))



195


In [3]:
#Setting up keys
pinecone_api_key = os.environ.get("pinecone_API")

pc = Pinecone(api_key=pinecone_api_key)

api_key = os.getenv("OPENAI_API_KEY")
openai.api_key = api_key


In [10]:
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

pc.create_index(
    name= 'department-abbrev-db',
    dimension=1536,
    metric="cosine",
    spec=ServerlessSpec(cloud="aws", region="us-east-1"),
)



PineconeApiException: (409)
Reason: Conflict
HTTP response headers: HTTPHeaderDict({'content-type': 'text/plain; charset=utf-8', 'access-control-allow-origin': '*', 'vary': 'origin,access-control-request-method,access-control-request-headers', 'access-control-expose-headers': '*', 'x-pinecone-api-version': '2024-07', 'X-Cloud-Trace-Context': '61ad522ff6857dfd325f71e34b875996', 'Date': 'Sat, 22 Feb 2025 22:46:59 GMT', 'Server': 'Google Frontend', 'Content-Length': '85', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'})
HTTP response body: {"error":{"code":"ALREADY_EXISTS","message":"Resource  already exists"},"status":409}


In [11]:
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

In [6]:
index = pc.Index("department-abbrev-db")

In [7]:
documents = [Document(page_content = doc) for doc in departments]
print(documents[0])

page_content='SWA'


In [12]:
vector_store = PineconeVectorStore(index=index, embedding=embeddings)
vector_store.add_documents(documents)



['f808f37d-b544-4dfd-8ab0-8d8082829164',
 '6c196248-5d91-4b00-8a6d-72d30f4dbc7d',
 '552344ee-c9a0-459d-a564-2efb2bc7229d',
 '8081bb13-ead1-40ff-8246-5c4672753732',
 'ea88e175-583d-4a96-a0cc-4f37e9e82055',
 '32b454e7-f1bc-4742-b23a-1cae2e808c90',
 '7eb0b49b-d730-4de0-bad3-e2758270b093',
 '55d1fe58-8689-4705-b149-e8c318725312',
 'ef825edd-67a4-46ff-aef9-6514ba633cc0',
 '5a43808e-180e-462f-995b-f0c4b69c724a',
 '63706fa6-594f-4025-998d-0b1be8fa5aef',
 '4db5f691-a4f8-4173-b743-abd47454999c',
 'd400f22f-e9cf-43b2-9c45-125d004f6387',
 '0a6f97f2-e918-4510-a9a1-c2679b62a73b',
 '7f70e1a8-9cd7-4351-b676-2313d5c56d10',
 '390bf1d5-7958-4eb8-907b-dc788fa1b15a',
 'cd8ea2dc-48bb-42a6-a8a0-b9e88521505f',
 '1f859aed-a964-42de-a3d5-0a02a654f252',
 'e09ebfe1-f0b8-4cbe-a083-59f5819f56d3',
 'a70fc283-ab7c-49ba-b2b1-98cf2169bb56',
 '555a9994-398c-4791-aff6-d306a7423c61',
 'cb28bf12-ef31-4448-a51b-7cb9add68c07',
 'c9ddbbc3-ecb3-4d2b-9c74-42c78b126cf0',
 '1fa1f71a-f1bc-449d-9562-2a2943b8589e',
 '4d1d8a7d-cb5e-

In [15]:
results = vector_store.similarity_search(
    "What are some computer engineering courses?",
    k=2,
)
for res in results:
    print(f"* {res.page_content} [{res.metadata}]")

* ECE [{}]
* CSE [{}]
